In [2]:
import json
import folium
from polygon_geohasher.polygon_geohasher import geohash_to_polygon
import geopandas as gpd
import numpy as np
import geohash
import pandas as pd

In [3]:
locations = pd.read_csv('buildings.csv')
locations = locations[['Y','X']]
locations.head()

,Y,X
0,13.091787,77.577790
1,13.091909,77.578081
2,13.092083,77.579872
3,13.092224,77.579709
4,13.092180,77.579897


In [4]:
locations = locations.values.tolist()
list = []
for i in locations:
    list.append((float("%14.12f"%float(i[0])),float("%15.13f"%float(i[1]))))
locations = list

In [5]:
# Create Geo Pandas DataFrame
df = gpd.GeoDataFrame({'location':locations, 'value':np.random.rand(1299)})
df['geohash'] = df['location'].apply(lambda l: geohash.encode(l[0], l[1], 7))
df.head()

,location,value,geohash
0,"(13.091787115417, 77.5777895255905)",0.150652,tdr4mr6
1,"(13.091909299642, 77.5780814805822)",0.014112,tdr4mr6
2,"(13.092082899557, 77.5798716415947)",0.949964,tdr4mr7
3,"(13.092223900217, 77.5797091578729)",0.900976,tdr4mr7
4,"(13.092180250625, 77.5798974667013)",0.007237,tdr4mr7


In [6]:
df['geometry'] = df['geohash'].apply(geohash_to_polygon)
df.head()

,location,value,geohash,geometry
0,"(13.091787115417, 77.5777895255905)",0.150652,tdr4mr6,"POLYGON ((77.57720947265625 13.09158325195312,..."
1,"(13.091909299642, 77.5780814805822)",0.014112,tdr4mr6,"POLYGON ((77.57720947265625 13.09158325195312,..."
2,"(13.092082899557, 77.5798716415947)",0.949964,tdr4mr7,"POLYGON ((77.57858276367188 13.09158325195312,..."
3,"(13.092223900217, 77.5797091578729)",0.900976,tdr4mr7,"POLYGON ((77.57858276367188 13.09158325195312,..."
4,"(13.092180250625, 77.5798974667013)",0.007237,tdr4mr7,"POLYGON ((77.57858276367188 13.09158325195312,..."


In [7]:
df.crs = {'init': 'epsg:4326'}

In [8]:
display(json.loads(df.to_json())['features'][0])
display(df.head())

{'id': '0',
 'type': 'Feature',
 'properties': {'location': [13.091787115417, 77.5777895255905],
  'value': 0.15065241682906727,
  'geohash': 'tdr4mr6'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[77.57720947265625, 13.091583251953125],
    [77.57858276367188, 13.091583251953125],
    [77.57858276367188, 13.09295654296875],
    [77.57720947265625, 13.09295654296875],
    [77.57720947265625, 13.091583251953125]]]}}

,location,value,geohash,geometry
0,"(13.091787115417, 77.5777895255905)",0.150652,tdr4mr6,"POLYGON ((77.57720947265625 13.09158325195312,..."
1,"(13.091909299642, 77.5780814805822)",0.014112,tdr4mr6,"POLYGON ((77.57720947265625 13.09158325195312,..."
2,"(13.092082899557, 77.5798716415947)",0.949964,tdr4mr7,"POLYGON ((77.57858276367188 13.09158325195312,..."
3,"(13.092223900217, 77.5797091578729)",0.900976,tdr4mr7,"POLYGON ((77.57858276367188 13.09158325195312,..."
4,"(13.092180250625, 77.5798974667013)",0.007237,tdr4mr7,"POLYGON ((77.57858276367188 13.09158325195312,..."


In [9]:
overlay = df.groupby('geohash').value.nunique()
overlay.head()

geohash
tdr4mr6      2
tdr4mr7     20
tdr4mrd      9
tdr4mre    101
tdr4mrg     69
Name: value, dtype: int64

In [10]:
lat, lng = (13.100211194785802,77.58364677429199) # 강남 Lat, Lng
m = folium.Map((lat, lng), zoom_start=15)
folium.Choropleth(geo_data=df, 
                  name='choropleth',
                  data=overlay,
                  columns=['geohash', 'value'],
                  key_on='feature.properties.geohash',
                  fill_color='YlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='Residential Areas in Yelahanka').add_to(m)

In [11]:
m.save('map.html')
m